In [ ]:
import numpy as np
import os
import sys
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, LeakyReLU
from keras import backend as K
import time


#for arrow classification
#not super robust trained on very few images
weight_file = 'Arrows_weights.h5'

def gen_class_model(file_name,labels=4):
    model = Sequential()
    #input a 64X64 img with 3 channels RGB use relu activation layer
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(labels, activation='softmax'))

    model.load_weights(file_name)
    # compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

model = gen_class_model('Arrows_weights.h5')

In [ ]:
from zumi.util.camera import Camera
import numpy as np
from PIL import Image
import IPython.display

camera = Camera(64, 64)
camera.start_camera()

try: 
    while True:
        #take a picture
        frame = camera.capture()
        
        #convert image
        img = Image.fromarray(frame)
        
        #turn image matrix into one column
        x = np.expand_dims(img, axis=0)
        
        #make a prediction based on input image
        preds = model.predict_classes(x)
        
        #return probabilities
        prob = model.predict_proba(x)
        
        print(preds, " prob", prob)
        
        #display what zumi sees
        IPython.display.display(img)
        IPython.display.clear_output(wait=True) 
finally:
    camera.close()
    print("done")